# **1. KURULUMLAR**

In [1]:
import numpy as np
import math
import re
import pandas as pd
from bs4 import BeautifulSoup
import random
from google.colab import drive


In [2]:
!pip install bert-for-tf2
!pip install sentencepiece

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 41.2/41.2 kB 1.3 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Preparing metadata (setup.py) ... done
  Preparing metadata (setup.py) ... done
  Created wheel for bert-for-tf2: filename=bert_for_tf2-0.14.9-py3-none-any.whl size=30510 sha256=5f17f6a761f57a4d11c830d60cc383e1ed09daa2531aacc3463a6162d8b428f7
  Stored in directory: /root/.cache/pip/wheels/d8/da/50/126d7b8416d9a0e6bf876935c2219a71e72a6529c25e150c56
  Created wheel for params-flow: filename=params_flow-0.8.2-py3-none-any.whl size=19455 sha256=f247b4e227c97eb72b7881bb0ecfb0e8eb2123d5f5f594d743f132a410516513
  Stored in directory: /root/.cache/pip/wheels/97/a8/d0/f7419404174976a2686bb98b5c30df01cc71445415f32db9e6
  Created wheel for py-params: filename=py_params-0.10.2-py3-none-any.whl size=7891 sha256=2529dbf6eb8885e38453cbec338f5f295d7bb36b115a4424036ff4a330d9084a
  Stored in directory: /root/.cache/pip/wheels/69/c8/b3/92666cff9fb312bc3473eaa6b396695b89a7b3e31e908768

In [3]:
try:
  %tensorflow_version 2.x
except Exception:
  pass

import tensorflow as tf

import tensorflow_hub as hub

from tensorflow.keras import layers

import bert

Colab only includes TensorFlow 2.x; %tensorflow_version has no effect.


# **2. Data Preprocessing**

## **Dosya Yükleme**


In [4]:

drive.mount("/content/drive")

Mounted at /content/drive


In [6]:
cols=["sentiment","id","date","query","user","text"]
data=pd.read_csv("/content/drive/MyDrive/BERT/train.csv",
                 header=None,
                 names=cols,
                 engine="python",
                 encoding="latin1")

In [7]:
data.drop(["id","date","query","user"],axis=1,inplace=True)

In [8]:
data.head(5)

,sentiment,text
0,0,"@switchfoot http://twitpic.com/2y1zl - Awww, t..."
1,0,is upset that he can't update his Facebook by ...
2,0,@Kenichan I dived many times for the ball. Man...
3,0,my whole body feels itchy and like its on fire
4,0,"@nationwideclass no, it's not behaving at all...."


## **Data Temizleme**

In [9]:
def clean_tweet(tweet):
  tweet= BeautifulSoup(tweet,"lxml").get_text()
  tweet= re.sub(r"@[A-Za-z-0-9]+"," ",tweet)
  tweet= re.sub(r"https?://+[A-Za-z-0-9./]+"," ",tweet)
  tweet= re.sub(r"[^a-zA-Z.!?']+"," ",tweet)
  tweet= re.sub(r" +"," ",tweet)
  return tweet


In [10]:
data_clean= [clean_tweet(tweet) for tweet in data.text]

<ipython-input-9-9854aae3979a>:2: MarkupResemblesLocatorWarning: The input looks more like a filename than markup. You may want to open this file and pass the filehandle into Beautiful Soup.
  tweet= BeautifulSoup(tweet,"lxml").get_text()


In [11]:
data_labels=data.sentiment.values
data_labels[data_labels==4]=1

## **Tokenization**

In [12]:
FullTokenizer=bert.bert_tokenization.FullTokenizer
bert_layer=hub.KerasLayer("https://tfhub.dev/tensorflow/bert_en_uncased_L-12_H-768_A-12/1",trainable=False)

vocab_file=bert_layer.resolved_object.vocab_file.asset_path.numpy() #tokenizer için kelime dosyasına erişim sağlıyor?
do_lower_case=bert_layer.resolved_object.do_lower_case.numpy() #case sensitive mi?
tokenizer=FullTokenizer(vocab_file,do_lower_case)


In [13]:
tokenizer.tokenize("My dog love strawberries.")

['my', 'dog', 'love', 'straw', '##berries', '.']

In [14]:
tokenizer.convert_tokens_to_ids(tokenizer.tokenize("My dog love strawberries."))

[2026, 3899, 2293, 13137, 20968, 1012]

In [15]:
def encode_sentence(sent):
  return tokenizer.convert_tokens_to_ids(tokenizer.tokenize(sent))

In [16]:
data_input=[encode_sentence(sentence) for sentence in data_clean]

In [17]:
print(data_input[0])

[22091, 2860, 2860, 2008, 1005, 1055, 1037, 26352, 5017, 1012, 2017, 2323, 2050, 2288, 2585, 12385, 1997, 2353, 2154, 2000, 2079, 2009, 1012, 1040]


## **Dataset Oluşturma**

In [18]:
data_with_len = []  # Boş bir liste oluşturuyoruz

# `data_inputs` listesi üzerinde döngü
for i, sent in enumerate(data_input):
    # Her bir cümle için, cümleyi, etiketini ve uzunluğunu içeren listeyi oluştur
    item = [sent, data_labels[i], len(sent)]

    # Oluşturulan listeyi `data_with_len` listesine ekle
    data_with_len.append(item)

random.shuffle(data_with_len)
data_with_len.sort(key=lambda x: x[2])

sorted_all = []  # Boş bir liste oluşturuyoruz

# `data_with_len` listesi üzerinde döngü
for sent_lab in data_with_len:
    # Eğer uzunluk 7'den büyükse, belirli formatta liste oluştur
    if sent_lab[2] > 7:
        # İki elemanlı tuple oluştur ve `sorted_all` listesine ekle
        item = (sent_lab[0], sent_lab[1])
        sorted_all.append(item)

TensorFlow'da bir veri kümesi (tf.data.Dataset) oluşturmak için kullanılır. Bu veri kümesi, sorted_all adlı bir listeyi TensorFlow veri formatına dönüştürür.

In [19]:
all_dataset=tf.data.Dataset.from_generator(lambda: sorted_all,output_types=(tf.int32,tf.int32))

In [20]:
next(iter(all_dataset))

(<tf.Tensor: shape=(8,), dtype=int32, numpy=array([4283, 4658, 2666, 2611, 1012, 1012, 8840, 2140], dtype=int32)>,
 <tf.Tensor: shape=(), dtype=int32, numpy=1>)

**all_dataset:** Bu, önceki adımlarda oluşturduğunuz TensorFlow veri kümesidir.

**padded_batch:** Bu metod, veri kümesini belirli bir batch boyutunda pad'leyerek batch'lere ayırır.

**BATCH_SIZE:** Her bir batch'in içereceği örnek sayısını belirler. Bu, genellikle bir değişken olarak tanımlanır ve burada sabit bir değeri temsil eder.

**padded_shapes:** Bu, her bir batch'in şeklinin nasıl pad'leneceğini belirtir.

**padded_shapes Açıklaması:**
((None,), (())): Bu, veri kümesindeki her öğenin (tuple) pad'lenme şekillerini tanımlar.

  *   (None,): Bu, dizinin ilk elemanının (örneğin, bir dizi veya cümle) dinamik uzunluğa sahip olduğunu belirtir. None değeri, TensorFlow'a bu boyutun dinamik olduğunu ve pad'lenmesi gerektiğini söyler.
  *    ((),): Bu, dizinin ikinci elemanının sabit bir boyuta sahip olduğunu belirtir. Bu durumda, sabit bir skalar değer (örneğin, bir etiket veya sınıflandırma) pad'lenmesine gerek yoktur.



In [21]:
BATCH_SIZE=32
all_batched=all_dataset.padded_batch(BATCH_SIZE,padded_shapes=((None,),(())))

In [22]:
next(iter(all_batched))

(<tf.Tensor: shape=(32, 8), dtype=int32, numpy=
 array([[ 4283,  4658,  2666,  2611,  1012,  1012,  8840,  2140],
        [ 2821,  2008,  1005,  1055, 18783,  3504,  2204,   999],
        [10261,  2017,  1037, 10392,  5353,   999,  2562,  5629],
        [ 8670,  2232,   999, 23042,  2000,  8915, 18886,  2015],
        [ 8909,  2243,  1045,  3335,  3331,  2000,  2017,  7887],
        [ 1045,  1005,  1049,  2012,  2147,  2035,  2894,   999],
        [ 3374,  2053, 16324,  2005,  2017,  2059,  1012,  1012],
        [ 2003,  9107,  2014,  2154,  2125,  1999,  1996,  3103],
        [ 2145,  2125,  5305,  2514,  2066, 13433,  2080,   999],
        [ 2821,  6203,   999,  2008,  1005,  1055,  2428, 28543],
        [ 2725,  2026,  2381,  2607,  6198,  2129,  4569,  1012],
        [ 2021,  2568,  2003,  2205,  2460,  2005,  2019,  2792],
        [ 2123,  1005,  1056,  2057,  2035,  1012,  8840,  2140],
        [ 2026,  2606,  3504,  2066,  2023,  2007,  2026, 21177],
        [ 1045,  2215,  2000

In [23]:
NB_BATCHES=math.ceil(len(sorted_all)/BATCH_SIZE)
NB_BATCHES_TEST=NB_BATCHES//10
all_batched.shuffle(NB_BATCHES) #datasetin büyük değilse batch sayısı kadar verebilirsin parametreyi
test_dataset=all_batched.take(NB_BATCHES_TEST)# all batcehd içinden ilk NB_BATCHES_TEST adetini alır.
train_dataset=all_batched.skip(NB_BATCHES_TEST)

# 3. Model Geliştirme

**DCNN Sınıfı:**
DCNN sınıfı, metin sınıflandırma için derin evrişimli sinir ağı (Deep Convolutional Neural Network) modelini tanımlar. Model, tf.keras.Model sınıfından türetilmiştir ve aşağıdaki yapı taşlarına sahiptir:

**Parametreler:**

- **vocab_size:**
  - **Anlamı:** Kelime haznesinin büyüklüğü (vocabulary size).
  - **Kullanımı:** Gömme katmanının (embedding layer) boyutunu belirlemek için kullanılır. Bu, modelin kaç farklı kelimeyi öğrenebileceğini gösterir.

- **embedding_dim (varsayılan: 128):**
  - **Anlamı:** Gömme (embedding) boyutu.
  - **Kullanımı:** Her kelimenin kaç boyutlu bir vektörle temsil edileceğini belirler. Örneğin, 128 boyutlu bir gömme vektörü, her kelimenin 128 boyutlu bir uzayda temsil edildiği anlamına gelir.

- **nb_filters (varsayılan: 50):**
  - **Anlamı:** Her evrişim (convolution) katmanındaki filtre sayısı.
  - **Kullanımı:** Evrişim katmanlarının derinliğini belirler. Daha fazla filtre, daha fazla özellik çıkarımı yapabilir.

- **FFN_units (varsayılan: 512):**
  - **Anlamı:** Tam bağlı katman (fully connected layer) birim sayısı.
  - **Kullanımı:** Evrişim katmanlarından sonra gelen tam bağlı katmanların büyüklüğünü belirler.

- **nb_classes (varsayılan: 2):**
  - **Anlamı:** Sınıf sayısı (örneğin, ikili sınıflandırma için 2).
  - **Kullanımı:** Çıkış katmanındaki nöron sayısını belirler. İkili sınıflandırma için genellikle 2, çok sınıflı sınıflandırma için sınıf sayısı kadar olur.

- **dropout_rate (varsayılan: 0.1):**
  - **Anlamı:** Dropout katmanındaki dropout oranı.
  - **Kullanımı:** Dropout, overfitting'i önlemek için kullanılır. Bu oran, her eğitim adımında nöronların belirli bir yüzdesinin rastgele olarak devre dışı bırakılacağını belirtir.

- **training (varsayılan: False):**
  - **Anlamı:** Modelin eğitim aşamasında olup olmadığını belirtir.
  - **Kullanımı:** Eğitim ve tahmin sırasında bazı katmanların davranışını kontrol eder (örneğin, dropout katmanı).

- **name (varsayılan: "dcnn"):**
  - **Anlamı:** Modelin adı.
  - **Kullanımı:** Modelin adını belirlemek için kullanılır. Bu, modelin kaydedilmesi veya yeniden yüklenmesi sırasında kullanılabilir.

**Yapı Taşları:**

- **self.embedding = layers.Embedding(vocab_size, embedding_dim):**
  - **Anlamı:** Gömme katmanı.
  - **Kullanımı:** Girdi kelimelerini belirli boyutlarda gömme vektörlerine dönüştürür. Bu, kelimelerin sürekli vektörler olarak temsil edilmesini sağlar.

- **self.bigram = layers.Conv1D(filters=nb_filters, kernel_size=2, padding="valid", activation="relu"):**
  - **Anlamı:** 1D evrişim (Convolution) katmanı.
  - **Kullanımı:** Tek boyutlu evrişim katmanı, metin verisi üzerinde yerel özellikleri çıkarmak için kullanılır. Bu katman, belirli bir penceredeki kelimeler arasındaki ilişkileri öğrenir.
  - **Parametreler:**
    - **filters=nb_filters:** Kullanılan filtre sayısını belirtir.
    - **kernel_size=2:** Evrişim penceresinin boyutunu belirtir. Burada 2 kelimelik bir pencere kullanılır.
    - **padding="valid":** Veri pad'lemesinin nasıl yapılacağını belirtir. "valid" padding, pad'leme yapmadan sadece geçerli evrişimleri kullanır.
    - **activation="relu":** Aktivasyon fonksiyonu olarak ReLU kullanılır. ReLU, doğrusal olmayan bir aktivasyon fonksiyonudur ve modele doğrusal olmayanlık ekler.

**Neden 1D Katman Kullanılır?**
- **Metin Verisi:** Metin verisi, sıralı ve tek boyutlu bir veri olduğundan, 1D evrişim katmanları kullanılır. Her kelime bir boyut olarak temsil edilir.
- **Yerel Bağlam:** 1D evrişim katmanları, metin içinde yerel bağlamı yakalamak için kullanılır. Bu, belirli pencerelerdeki kelimeler arasındaki ilişkileri öğrenir.
- **Verimli Hesaplama:** 1D evrişimler, 2D evrişimlere göre daha az hesaplama gerektirir ve metin verileri için daha uygundur.


**GlobalMaxPool1D Katmanı:**
**GlobalMaxPool1D** katmanı, 1D evrişim katmanlarından çıkan özellik haritalarının maksimum değerlerini alır ve boyutlarını küçültür. Bu, önemli özelliklerin özetlenmesini sağlar. Parametre olarak genellikle özel bir ayar yapmaya gerek yoktur.

**Dense (Tam Bağlı) Katmanı:**
**Dense** katmanı, giriş özelliklerini ağırlıklarla çarpar ve bir aktivasyon fonksiyonu (bu örnekte ReLU) uygulayarak çıkış üretir. `units=FFN_units` bu katmandaki nöron sayısını belirtir. ReLU aktivasyon fonksiyonu doğrusal olmayanlık ekler ve modelin daha karmaşık ilişkileri öğrenmesine yardımcı olur.

**Dropout Katmanı:**
**Dropout** katmanı, eğitim sırasında rastgele olarak belirli bir yüzdede nöronu devre dışı bırakır (bu oran `dropout_rate` ile belirlenir). Bu, overfitting'i önlemeye yardımcı olur ve modelin genelleme yeteneğini artırır.

**call Fonksiyonu:**

- **Amaç:** `call` fonksiyonu, modelin öngörü ve eğitim sırasında nasıl işlem yapacağını tanımlar. Bu, modelin öngörülerini yapmak ve geri yayılım aşamasında hesaplamalar yapmak için kullanılır.

- **`tf.concat([x_1, x_2, x_3], axis=-1)`:** Burada `x_1`, `x_2`, ve `x_3` değişkenleri, farklı evrişim katmanlarından çıkan ve ardından havuzlama (pooling) uygulanan özellik haritalarını temsil eder. `tf.concat` fonksiyonu, bu özellik haritalarını belirli bir eksende birleştirir. `axis=-1` kullanımı, birleştirmenin son eksende (yani özellik ekseninde) yapıldığını belirtir. Bu, çeşitli evrişim pencerelerinden gelen bilgilerin bir araya getirilmesini sağlar.

- **Sonraki Adımlar:** Birleştirilen özellik haritaları, tam bağlı (dense) bir katmanda işlenir, dropout uygulanır ve son çıkış tahmini yapılır.


**Özellik Haritaları Nedir?**

**Özellik haritaları** (feature maps), evrişimli sinir ağlarında (Convolutional Neural Networks - CNN) bulunan bir ara sonuç türüdür. Bu haritalar, evrişim katmanlarının girdi verisi üzerinde uyguladığı filtrelerin (kernels) sonucunda elde edilir.

**Nasıl Çalışır?**

1. **Evrişim Katmanı:**
   - Bir evrişim katmanı, belirli bir pencerede (örneğin, 2 kelime, 3 kelime) veri üzerinde bir filtre uygular.
   - Bu filtre, veri üzerinde kaydırılarak belirli özellikleri (örneğin, kelime çiftleri veya üçlüleri) öğrenir.

2. **Özellik Haritası Oluşumu:**
   - Filtreler, verinin her bölgesinde (pencere) uygulandığında, her bir bölge için bir değer üretilir.
   - Bu değerler, bir matris (özellik haritası) oluşturur. Her bir matris elemanı, filtre uygulamasının sonucudur.

   Örneğin, 2 kelimelik bir filtre uygulandığında, her iki kelimenin bir arada olduğu bölgelerdeki ilişkileri yakalayabiliriz. Filtre kaydırıldıkça, metin üzerindeki tüm bölgeleri kapsayan bir özellik haritası elde edilir.

**Özellik Haritalarının Önemi**

- **Yerel Özellikler:** Özellik haritaları, metindeki belirli pencerelerdeki yerel özellikleri öğrenir. Bu, metin verisindeki belirli desenleri veya ilişkileri yakalamak için kullanılır.
- **Özelleştirilmiş Temsiller:** Farklı filtreler, farklı özellikleri öğrenir ve her biri farklı bir özellik haritası oluşturur. Örneğin, bir filtre kelime çiftlerini öğrenirken, başka bir filtre kelime üçlülerini öğrenebilir.

**Havuzlama (Pooling) ile Özellik Haritalarını Kullanma**

- **Pooling:** Havuzlama, özellik haritalarının boyutunu küçültür ve önemli bilgileri özetler. Örneğin, maksimum havuzlama (Max Pooling) yöntemi, özellik haritasındaki her pencerede en yüksek değeri seçer.
- **Özetleme:** Bu özetlenmiş bilgiler, modelin daha ileri aşamalarında kullanılır. Havuzlama, önemli özelliklerin korunmasını sağlar ve gereksiz ayrıntıları azaltır.

**Sonuç**

- **Özellik Haritaları:** Evrişim katmanlarından elde edilen, verinin belirli bölgelerinde öğrenilen özelliklerin matrisleridir.
- **Havuzlama:** Özellik haritalarının boyutunu küçültüp önemli bilgileri özetler.
- **Dense Katmanlar:** Özellik haritalarını işleyerek nihai tahminler yapar.

Özetle, özellik haritaları, evrişim katmanlarının veriyi nasıl öğrendiğini gösterir ve havuzlama ile bu bilgilerin önemli özetlerini elde ederiz. Bu özetler, modelin son tahminlerini yapmak için kullanılır.


**Not:** Embedder veriyi sayısal bir forma dönüştüren bir araçtır, Transformer ise bu veriyi işleyip anlamlı sonuçlar çıkarmak için kullanılan bir model mimarisidir. Embedding genellikle Transformer gibi daha karmaşık modellerin bir parçası olarak kullanılır.

In [44]:
class DCNN(tf.keras.Model):
  def __init__(self,
               vocab_size,
               emb_dim=128,
               nb_filters=50,
               FFN_units=512,
               nb_classes=2,
               dropout_rate=0.1,
               training=False,
               name="dcnn"):
    super(DCNN,self).__init__(name=name)
    self.embedding=layers.Embedding(vocab_size,emb_dim)

    self.biagram=layers.Conv1D(filters=nb_filters,
                              kernel_size=2,
                              padding="valid",
                              activation="relu")


    self.trigram=layers.Conv1D(filters=nb_filters,
                              kernel_size=3,
                              padding="valid",
                              activation="relu")


    self.fourtgram=layers.Conv1D(filters=nb_filters,
                              kernel_size=4,
                              padding="valid",
                              activation="relu")

    self.pool=layers.GlobalMaxPool1D()

    self.dense_1=layers.Dense(units=FFN_units,
                              activation="relu")

    self.dropout=layers.Dropout(rate=dropout_rate)

    if nb_classes==2:
      self.last_dense=layers.Dense(units=1,
                                   activation="sigmoid")
    else:
      self.last_dense=layers.Dense(units=nb_classes,
                                   activation="softmax")

  def call(self,inputs,training):
    x = self.embedding(inputs)

    x_1=self.biagram(x)
    x_1=self.pool(x_1)

    x_2=self.trigram(x)
    x_2=self.pool(x_2)

    x_3=self.trigram(x)
    x_3=self.pool(x_3) # (batc_Size,nb_filters)

    merged=tf.concat([x_1,x_2,x_3],axis=-1) #(batch_size, 3*nb_filters)
    merged=self.dense_1(merged)
    merged=self.dropout(merged, training)
    output=self.last_dense(merged)

    return output

# 4.Model Building

In [45]:
VOCAB_SIZE=len(tokenizer.vocab)
EMB_DIM=200
NB_FILTERS=100
FFN_UNITS=256
NB_CLASSES=2
DROPOUT_TATE=0.2
NB_EPOCHS=5

In [46]:
Dcnn=DCNN(vocab_size=VOCAB_SIZE,
          emb_dim=EMB_DIM,
          nb_filters=NB_FILTERS,
          FFN_units=FFN_UNITS,
          nb_classes=NB_CLASSES,
          dropout_rate=DROPOUT_TATE)

In [47]:
if NB_CLASSES==2:
  Dcnn.compile(loss="binary_crossentropy", #2 CLASS VAR O YÜZDEN
              optimizer="adam",
              metrics=["accuracy"])
else:
  Dcnn.compile(loss="sparse_categorical_crossentropy",
              optimizer="adam",
              metrics=["sparse_categorical_accuracy"])

In [48]:
#ağırlıkları korumak için checkpoint kullan.

checkpoint_path="/content/drive/MyDrive/BERT/ckpt_bert_tok"

ckpt= tf.train.Checkpoint(Dcnn=Dcnn)

ckpt_manager=tf.train.CheckpointManager(ckpt,checkpoint_path,max_to_keep=1) #5  DE YAPABİLİRSİN

if ckpt_manager.latest_checkpoint:
  ckpt.restore(ckpt_manager.latest_checkpoint)
  print("Lastest Checkpoint Restored from {}".format(ckpt_manager.latest_checkpoint))

In [49]:
class MyCustomCallback(tf.keras.callbacks.Callback):
  #her epokta kaydetmeye yarıyor?
  def on_epoch_end(self, epoch, logs=None):
    ckpt_manager.save()
    print("Saving checkpoint for epoch {} at {}".format(epoch+1,
                                                        checkpoint_path))

In [50]:
Dcnn.fit(train_dataset,
        epochs=NB_EPOCHS,
        callbacks=[MyCustomCallback()])

Epoch 1/5
  37193/Unknown - 479s 12ms/step - loss: 0.4327 - accuracy: 0.8001Saving checkpoint for epoch 1 at /content/drive/MyDrive/BERT/ckpt_bert_tok
37193/37193 [==============================] - 479s 12ms/step - loss: 0.4327 - accuracy: 0.8001
Epoch 2/5
37193/37193 [==============================] - 403s 10ms/step - loss: 0.3873 - accuracy: 0.8269
Epoch 3/5
37193/37193 [==============================] - 399s 10ms/step - loss: 0.3512 - accuracy: 0.8462
Epoch 4/5
37193/37193 [==============================] - 401s 10ms/step - loss: 0.3154 - accuracy: 0.8645
Epoch 5/5
37193/37193 [==============================] - 402s 10ms/step - loss: 0.2819 - accuracy: 0.8799


# 5. Evaluation

In [51]:
result=Dcnn.evaluate(test_dataset)
print(result)

4132/4132 [==============================] - 36s 9ms/step - loss: 0.4212 - accuracy: 0.8293
[0.4212311804294586, 0.82933509349823]


In [59]:
def get_prediction(sentence):
  tokens=encode_sentence(sentence)
  inputs=tf.expand_dims(tokens,0)
  output=Dcnn(inputs,training=False)
  sentiment=math.floor(output*2)
  if sentiment == 0:
        print(f"Output of the model:{output}. Predicted sentiment negative.")
  else:
        print(f"Output of the model:{output}. Predicted sentiment positive.")



In [53]:
get_prediction("I'd rather not do that again.")

Output of the model:[[0.22451645]]. Predicted sentiment negative.


In [54]:
get_prediction("This movie was pretty interesting.")

Output of the model:[[0.96834356]]. Predicted sentiment positive.


In [55]:
get_prediction("I am falling love to this car.")

Output of the model:[[0.88105756]]. Predicted sentiment positive.


In [61]:
get_prediction("I dont know sometimes ı am hopeless.")

Output of the model:[[0.15894072]]. Predicted sentiment negative.
